In [ ]:
from tabulate import tabulate
import copy

In [ ]:
class GridEnv:

  def __init__(self, M, N, goal, hazard, obstacles, r,  max_iterations = 100, intended_action_probability = 0.8, gamma = 0.9, epsilon = 1e-08,):
    '''
      Initializes the Grid Environment of size M x N with the given environment properties
    '''
    self.M = M
    self.N = N
    self.goal = goal
    self.hazard = hazard
    self.obstacles = obstacles
    self.r = r
    self.γ = gamma
    self.ε = epsilon
    self.actions = {"L" : "←", "U" : "↑", "D" : "↓", "R" : "→"}
    self.opp_actions = {"L" : "R", "R" : "L", "U" : "D", "D" : "U"}
    self.max_iterations = max_iterations
    self.intended_action_probability = intended_action_probability

  def generate_new_grid(self):
    '''
      Generates a new Grid of size M x N with filled obstacles, Destinations and Hazards
    '''
    return [[0 for i in range(self.N)] for i in range(self.M)]

  def T(self, s, a):
    '''
      Returns the Next state based on the Action taken from the current state
    '''
    i, j = s
    if a == 'U':
      s1 = (max(0, i - 1), j)
    elif a == 'D':
      s1 = (min(self.M - 1, i + 1), j)
    elif a == 'L':
      s1 = (i, max(0, j - 1))
    elif a == 'R':
      s1 = (i, min(self.N - 1, j + 1))
    else:
      s1 = s

    if(s1 in self.obstacles):
      return s
    return s1

  def n_dest(self, s, ia):
    '''
      Returns the number of destinations that an Agent can reach from the current state
    '''
    i, j = s
    count = 0
    for a in self.actions:
      s1 = self.T(s, a)
      #print(s, s1, ia, a, self.opp_actions[ia])
      if  a != self.opp_actions[ia] and s1 not in self.obstacles:
        count += 1
    #print("Dests :", count)
    return count

  def trans_prob(self, s, ia, a):
    '''
      Returns the Probability of the action from the current state
    '''
    new_state = self.T(s, a)
    if(new_state not in self.obstacles and ia == a):
      p = self.intended_action_probability
    elif(new_state in self.obstacles or a == self.opp_actions[ia]):
      p = 0.0
    else:
      p = round((1 - self.intended_action_probability) / (self.n_dest(s, a) - 1), 2)
    return p

  def display_grid(self, grid):
    '''
      Display's the Grid
    '''
    print(tabulate(grid, tablefmt='grid'))

  def check_delta(self, Uk, Uk_p_1):
    '''
      Checks whether the Delta(U_k+1, U_k) for all states is lessthan ε
    '''
    for i in range(self.M):
      for j in range(self.N):
        if((i, j) not in self.obstacles):
          if(abs(Uk_p_1[i][j] - Uk[i][j]) > self.ε):
            return False
    return True

  def get_max_action(self, Uk, s, expr):
    '''
      Returns the maximum reward with the action when Agent moves from current state to next state
    '''
    values = {}
    for intended_action in self.actions:
      val = 0
      expr += "["
      for other_action in self.actions:
        if(other_action is not self.opp_actions[intended_action]):
          i, j = self.T(s, other_action)
          prob = self.trans_prob(s, intended_action, other_action)
          expr += f" ({prob} * {Uk[i][j]}) +"
          val += prob * Uk[i][j]
      values[round(val, 2)] = intended_action
      expr = expr[:-1] + " ], \n"
    # print(f"State - {s}, probability * reward - {values}, max reward - {values.keys()} with Action - {values[max(values.keys())]}")
    return max(values.keys()), values[max(values.keys())],expr

  def value_iteration(self, all_iterations = False, print_expr = False):
    '''
      Value iteration will runs until the max iterations reached or |U_k+1, Uk| < ε by employing the Bellmanford's Equation
      and returns the Utility list and Actions list
    '''
    Uk = self.generate_new_grid()
    A = self.generate_new_grid()
    self.display_grid(Uk)
    for i in self.goal:
        Uk[i[0]][i[1]] = self.goal[i]
        A[i[0]][i[1]] = 'D'
    for i in self.hazard:
        Uk[i[0]][i[1]] = self.hazard[i]
        A[i[0]][i[1]] = 'H'
    for i in self.obstacles:
        Uk[i[0]][i[1]] = 'B'
        A[i[0]][i[1]] = 'B'
    self.display_grid(Uk)
    Uk_p_1= copy.deepcopy(Uk)
    print("Running Iterations")
    for k in range(1, self.max_iterations + 1):
      print(k, end = "\n" if k % 10 == 0 else "-")
      for i in range(self.M):
        for j in range(self.N):
          s = (i, j)
          if s in self.obstacles + list(self.goal.keys()) + list(self.hazard.keys()):
            continue
          expr = f"V{s} = {self.r} + {self.γ} * max("
          max_val, action, in_expr = self.get_max_action(Uk, s, expr)
          Uk_p_1[i][j] = round(self.r + self.γ * max_val, 2)
          if(action != None):
            A[i][j] = self.actions[action]
          if(print_expr):
            print(f"{expr} {in_expr} ) = {Uk_p_1[i][j]}")
      if(self.check_delta(Uk, Uk_p_1) and not all_iterations):
        print(f"\nAt the iteration - {k} the |U_k+1 - Uk| > ε for all states")
        break
      if(print_expr):
        print(f"At Iteration {k}")
        self.display_grid(Uk_p_1)
        self.display_grid(A)
      Uk = copy.deepcopy(Uk_p_1)
    return Uk_p_1, A

In [ ]:
# Test Run for verification (Example discussed in class)
env = GridEnv(3, 4, {(0, 3) : 1}, {(1, 3) : -1}, [(1, 1)], 0, 100, 0.8)
# Set print_expr to True for displaying the Expressions
utility1, policy1 = env.value_iteration(print_expr=False)
env.display_grid(utility1)
env.display_grid(policy1)

+---+---+---+---+
| 0 | 0 | 0 | 0 |
+---+---+---+---+
| 0 | 0 | 0 | 0 |
+---+---+---+---+
| 0 | 0 | 0 | 0 |
+---+---+---+---+
+---+---+---+----+
| 0 | 0 | 0 |  1 |
+---+---+---+----+
| 0 | B | 0 | -1 |
+---+---+---+----+
| 0 | 0 | 0 |  0 |
+---+---+---+----+
Running Iterations
1-2-3-4-5-6-7-8-9-10
11-12-13-
At the iteration - 13 the |U_k+1 - Uk| > ε for all states
+------+------+------+-------+
| 0.65 | 0.75 | 0.85 |  1    |
+------+------+------+-------+
| 0.57 | B    | 0.58 | -1    |
+------+------+------+-------+
| 0.5  | 0.44 | 0.49 |  0.29 |
+------+------+------+-------+
+---+---+---+---+
| → | → | → | D |
+---+---+---+---+
| ↑ | B | ↑ | H |
+---+---+---+---+
| ↑ | ← | ↑ | ← |
+---+---+---+---+


In [ ]:
env = GridEnv(3, 4, {(0, 3) : 1}, {(1, 3) : -1}, [(1, 1)], 0, 100, 0.8)
# Set print_expr to True for displaying the Expressions
utility1, policy1 = env.value_iteration(all_iterations=True, print_expr=False)
env.display_grid(utility1)
env.display_grid(policy1)

+---+---+---+---+
| 0 | 0 | 0 | 0 |
+---+---+---+---+
| 0 | 0 | 0 | 0 |
+---+---+---+---+
| 0 | 0 | 0 | 0 |
+---+---+---+---+
+---+---+---+----+
| 0 | 0 | 0 |  1 |
+---+---+---+----+
| 0 | B | 0 | -1 |
+---+---+---+----+
| 0 | 0 | 0 |  0 |
+---+---+---+----+
Running Iterations
1-2-3-4-5-6-7-8-9-10
11-12-13-14-15-16-17-18-19-20
21-22-23-24-25-26-27-28-29-30
31-32-33-34-35-36-37-38-39-40
41-42-43-44-45-46-47-48-49-50
51-52-53-54-55-56-57-58-59-60
61-62-63-64-65-66-67-68-69-70
71-72-73-74-75-76-77-78-79-80
81-82-83-84-85-86-87-88-89-90
91-92-93-94-95-96-97-98-99-100
+------+------+------+-------+
| 0.65 | 0.75 | 0.85 |  1    |
+------+------+------+-------+
| 0.57 | B    | 0.58 | -1    |
+------+------+------+-------+
| 0.5  | 0.44 | 0.49 |  0.29 |
+------+------+------+-------+
+---+---+---+---+
| → | → | → | D |
+---+---+---+---+
| ↑ | B | ↑ | H |
+---+---+---+---+
| ↑ | ← | ↑ | ← |
+---+---+---+---+


##### Task - T1

```
# Genarating Policy P1 with R1 = 10, R2 = -5, r = -5
```

In [ ]:
env = GridEnv(5, 5, {(0, 3) : 10}, {(2, 2) : -5}, [(0, 0), (0, 4), (4, 0), (4, 4)], -5, 500, 0.9)
utility1, policy1 = env.value_iteration()
env.display_grid(utility1)
env.display_grid(policy1)

+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
+---+---+----+----+---+
| B | 0 |  0 | 10 | B |
+---+---+----+----+---+
| 0 | 0 |  0 |  0 | 0 |
+---+---+----+----+---+
| 0 | 0 | -5 |  0 | 0 |
+---+---+----+----+---+
| 0 | 0 |  0 |  0 | 0 |
+---+---+----+----+---+
| B | 0 |  0 |  0 | B |
+---+---+----+----+---+
Running Iterations
1-2-3-4-5-6-7-8-9-10

At the iteration - 10 the |U_k+1 - Uk| > ε for all states
+--------+--------+--------+--------+--------+
| B      |  -2.96 |   3.12 |  10    | B      |
+--------+--------+--------+--------+--------+
| -12.51 |  -7.77 |  -2.7  |   2.83 | -3.22  |
+--------+--------+--------+--------+--------+
| -14.52 | -10.05 |  -5    |  -3.3  | -8.12  |
+--------+--------+--------+--------+--------+
| -18.14 | -14.41 | -10.09 |  -8.69 | -12.53 |
+--------+--------+

In [ ]:
env = GridEnv(5, 5, {(0, 3) : 10}, {(2, 2) : -5}, [(0, 0), (0, 4), (4, 0), (4, 4)], -5, 500, 0.9)
utility1, policy1 = env.value_iteration(all_iterations=True)
env.display_grid(utility1)
env.display_grid(policy1)

+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
+---+---+----+----+---+
| B | 0 |  0 | 10 | B |
+---+---+----+----+---+
| 0 | 0 |  0 |  0 | 0 |
+---+---+----+----+---+
| 0 | 0 | -5 |  0 | 0 |
+---+---+----+----+---+
| 0 | 0 |  0 |  0 | 0 |
+---+---+----+----+---+
| B | 0 |  0 |  0 | B |
+---+---+----+----+---+
Running Iterations
1-2-3-4-5-6-7-8-9-10
11-12-13-14-15-16-17-18-19-20
21-22-23-24-25-26-27-28-29-30
31-32-33-34-35-36-37-38-39-40
41-42-43-44-45-46-47-48-49-50
51-52-53-54-55-56-57-58-59-60
61-62-63-64-65-66-67-68-69-70
71-72-73-74-75-76-77-78-79-80
81-82-83-84-85-86-87-88-89-90
91-92-93-94-95-96-97-98-99-100
101-102-103-104-105-106-107-108-109-110
111-112-113-114-115-116-117-118-119-120
121-122-123-124-125-126-127-128-129-130
131-132-133-134-135-136-137-138-139-140
141-142-143-144-145-146

#### T2


```
# 1. R1 = 50, R2 = -50, r = -5
```



In [ ]:
env = GridEnv(5, 5, {(0, 3) : 50}, {(2, 2) : -50}, [(0, 0), (0, 4), (4, 0), (4, 4)], -5, 500, 0.9)
utility2, policy2 = env.value_iteration()
env.display_grid(utility2)
env.display_grid(policy2)

+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
+---+---+-----+----+---+
| B | 0 |   0 | 50 | B |
+---+---+-----+----+---+
| 0 | 0 |   0 |  0 | 0 |
+---+---+-----+----+---+
| 0 | 0 | -50 |  0 | 0 |
+---+---+-----+----+---+
| 0 | 0 |   0 |  0 | 0 |
+---+---+-----+----+---+
| B | 0 |   0 |  0 | B |
+---+---+-----+----+---+
Running Iterations
1-2-3-4-5-6-7-8-9-10
11-12-13-14-15-
At the iteration - 15 the |U_k+1 - Uk| > ε for all states
+-------+-------+--------+-------+-------+
| B     | 28.39 |  38.53 | 50    | B     |
+-------+-------+--------+-------+-------+
| 12.02 | 20.04 |  28.82 | 38.06 | 27.97 |
+-------+-------+--------+-------+-------+
| 5.4   |  9.23 | -50    | 24.47 | 19.64 |
+-------+-------+--------+-------+-------+
| -0.53 |  2.7  |   5.48 | 15.61 | 12.15 |
+-------+-------+--------

In [ ]:
env = GridEnv(5, 5, {(0, 3) : 50}, {(2, 2) : -50}, [(0, 0), (0, 4), (4, 0), (4, 4)], -5, 500, 0.9)
utility2, policy2 = env.value_iteration(all_iterations=True)
env.display_grid(utility2)
env.display_grid(policy2)

+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
+---+---+-----+----+---+
| B | 0 |   0 | 50 | B |
+---+---+-----+----+---+
| 0 | 0 |   0 |  0 | 0 |
+---+---+-----+----+---+
| 0 | 0 | -50 |  0 | 0 |
+---+---+-----+----+---+
| 0 | 0 |   0 |  0 | 0 |
+---+---+-----+----+---+
| B | 0 |   0 |  0 | B |
+---+---+-----+----+---+
Running Iterations
1-2-3-4-5-6-7-8-9-10
11-12-13-14-15-16-17-18-19-20
21-22-23-24-25-26-27-28-29-30
31-32-33-34-35-36-37-38-39-40
41-42-43-44-45-46-47-48-49-50
51-52-53-54-55-56-57-58-59-60
61-62-63-64-65-66-67-68-69-70
71-72-73-74-75-76-77-78-79-80
81-82-83-84-85-86-87-88-89-90
91-92-93-94-95-96-97-98-99-100
101-102-103-104-105-106-107-108-109-110
111-112-113-114-115-116-117-118-119-120
121-122-123-124-125-126-127-128-129-130
131-132-133-134-135-136-137-138-139-140
141-142-143-

#### T3

```
# 2. R1 = 100, R2 = -500, r = -5
```



In [ ]:
env = GridEnv(5, 5, {(0, 3) : 100}, {(2, 2) : -500}, [(0, 0), (0, 4), (4, 0), (4, 4)], -5, 500, 0.9)
utility3, policy3 = env.value_iteration()
env.display_grid(utility3)
env.display_grid(policy3)

+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
+---+---+------+-----+---+
| B | 0 |    0 | 100 | B |
+---+---+------+-----+---+
| 0 | 0 |    0 |   0 | 0 |
+---+---+------+-----+---+
| 0 | 0 | -500 |   0 | 0 |
+---+---+------+-----+---+
| 0 | 0 |    0 |   0 | 0 |
+---+---+------+-----+---+
| B | 0 |    0 |   0 | B |
+---+---+------+-----+---+
Running Iterations
1-2-3-4-5-6-7-8-9-10
11-12-13-14-15-16-17-18-19-20
21-22-23-
At the iteration - 23 the |U_k+1 - Uk| > ε for all states
+-------+-------+---------+--------+-------+
| B     | 67.57 |   82.8  | 100    | B     |
+-------+-------+---------+--------+-------+
| 42.69 | 54.72 |   68.22 |  82.08 | 66.9  |
+-------+-------+---------+--------+-------+
| 32.12 | 24.21 | -500    |  43.54 | 53.55 |
+-------+-------+---------+--------+-------+
| 22.77 

In [ ]:
env = GridEnv(5, 5, {(0, 3) : 100}, {(2, 2) : -500}, [(0, 0), (0, 4), (4, 0), (4, 4)], -5, 500, 0.9)
utility3, policy3 = env.value_iteration(all_iterations=True)
env.display_grid(utility3)
env.display_grid(policy3)

+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
+---+---+------+-----+---+
| B | 0 |    0 | 100 | B |
+---+---+------+-----+---+
| 0 | 0 |    0 |   0 | 0 |
+---+---+------+-----+---+
| 0 | 0 | -500 |   0 | 0 |
+---+---+------+-----+---+
| 0 | 0 |    0 |   0 | 0 |
+---+---+------+-----+---+
| B | 0 |    0 |   0 | B |
+---+---+------+-----+---+
Running Iterations
1-2-3-4-5-6-7-8-9-10
11-12-13-14-15-16-17-18-19-20
21-22-23-24-25-26-27-28-29-30
31-32-33-34-35-36-37-38-39-40
41-42-43-44-45-46-47-48-49-50
51-52-53-54-55-56-57-58-59-60
61-62-63-64-65-66-67-68-69-70
71-72-73-74-75-76-77-78-79-80
81-82-83-84-85-86-87-88-89-90
91-92-93-94-95-96-97-98-99-100
101-102-103-104-105-106-107-108-109-110
111-112-113-114-115-116-117-118-119-120
121-122-123-124-125-126-127-128-129-130
131-132-133-134-135-136-137-13

#### T3


```
# 3. R1 = 50, R2 = -50, r = -1
```



In [ ]:
env = GridEnv(5, 5, {(0, 3) : 50}, {(2, 2) : -50}, [(0, 0), (0, 4), (4, 0), (4, 4)], -1, 500, 0.9)
utility5, policy5 = env.value_iteration()
env.display_grid(utility5)
env.display_grid(policy5)

+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
+---+---+-----+----+---+
| B | 0 |   0 | 50 | B |
+---+---+-----+----+---+
| 0 | 0 |   0 |  0 | 0 |
+---+---+-----+----+---+
| 0 | 0 | -50 |  0 | 0 |
+---+---+-----+----+---+
| 0 | 0 |   0 |  0 | 0 |
+---+---+-----+----+---+
| B | 0 |   0 |  0 | B |
+---+---+-----+----+---+
Running Iterations
1-2-3-4-5-6-7-8-9-10
11-12-13-14-15-
At the iteration - 15 the |U_k+1 - Uk| > ε for all states
+-------+-------+--------+-------+-------+
| B     | 37.03 |  43.12 | 50    | B     |
+-------+-------+--------+-------+-------+
| 27.13 | 31.94 |  37.3  | 42.83 | 36.77 |
+-------+-------+--------+-------+-------+
| 23.07 | 23.66 | -50    | 32.87 | 31.69 |
+-------+-------+--------+-------+-------+
| 19.46 | 19.94 |  20.11 | 27.75 | 27.14 |
+-------+-------+--------

In [ ]:
env = GridEnv(5, 5, {(0, 3) : 100}, {(2, 2) : -500}, [(0, 0), (0, 4), (4, 0), (4, 4)], -1, 500, 0.9)
utility4, policy4 = env.value_iteration()
env.display_grid(utility4)
env.display_grid(policy4)

+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+
+---+---+------+-----+---+
| B | 0 |    0 | 100 | B |
+---+---+------+-----+---+
| 0 | 0 |    0 |   0 | 0 |
+---+---+------+-----+---+
| 0 | 0 | -500 |   0 | 0 |
+---+---+------+-----+---+
| 0 | 0 |    0 |   0 | 0 |
+---+---+------+-----+---+
| B | 0 |    0 |   0 | B |
+---+---+------+-----+---+
Running Iterations
1-2-3-4-5-6-7-8-9-10
11-12-13-14-15-16-17-18-19-20

At the iteration - 20 the |U_k+1 - Uk| > ε for all states
+-------+-------+---------+--------+-------+
| B     | 76.21 |   87.38 | 100    | B     |
+-------+-------+---------+--------+-------+
| 57.97 | 66.79 |   76.69 |  86.86 | 75.73 |
+-------+-------+---------+--------+-------+
| 50.21 | 44.41 | -500    |  58.6  | 65.94 |
+-------+-------+---------+--------+-------+
| 43.35 | 38.42 |